# Gallery Example: M/M/1 Linear Tandem Network

This example demonstrates a linear (tandem) network of M/M/1 queues:
- **Topology**: n queues in series (tandem)
- **Arrivals**: Poisson process to first queue
- **Service**: Exponential service times with symmetric load profile
- **Load Profile**: Service times increase to middle, then decrease
- **Scheduling**: FCFS at all stations

This creates a bottleneck in the middle of the network, useful for studying load balancing effects.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_linear(n=2, Umax=0.9): Network {    """Create linear tandem network of M/M/1 queues        Parameters:    - n: Number of queues in the tandem (default 2)    - Umax: Maximum utilization at bottleneck (default 0.9)    """    model = Network("M/M/1-Linear")        # Block 1: nodes    line = []    line.append(Source(model, "mySource"))        for i in range(n):        queue = Queue(model, f"Queue{i+1}", SchedStrategy.FCFS)        line.append(queue)        line.append(Sink(model, "mySink"))        # Block 2: classes    oclass = OpenClass(model, "myClass")    line[0].setArrival(oclass, Exp(1))  # λ = 1        # Create symmetric load profile: increases to middle, then decreases    means = np.linspace(0.1, Umax, n//2)    if n % 2 == 0:        means = np.concatenate([means, means[::-1]])    else:        means = np.concatenate([means, [Umax], means[::-1]])        for i in range(n):        line[i+1].setService(oclass, Exp.fitMean(means[i]))        # Block 3: topology - serial routing    P = model.initRoutingMatrix()    for i in range(len(line)-1):        P.addRoute(oclass, line[i], line[i+1], 1.0)    model.link(P)        return model, means# Create the model with default parametersn_queues = 4max_util = 0.9model, service_means = gallery_mm1_linear(n_queues, max_util)println(f"Number of queues: {n_queues}")println(f"Service means: {service_means}")println(f"Utilizations: {1.0 * service_means}")

## Theoretical Analysis

For a tandem network:
- **Throughput**: Same at all stations (λ = 1)
- **Utilization**: ρᵢ = λ × service_time_i
- **Response Time**: Sum of individual response times
- **Bottleneck**: Station with highest utilization limits throughput

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 10)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)

In [ ]:
// Analyze bottleneck effects with different network sizes
println("\n=== Network Size Analysis ===")
for n in [2, 3, 5, 7]:
    model_n, means_n = gallery_mm1_linear(n, 0.8)
    val solver = MVA(model_n)
    val avg_table = solver.avgTable
// Calculate total response time (sum across all queues)
    val total_resp_time = 0
    val max_util = 0
    
    for i in range(1, n+1):  # Skip source (index 0) and sink (last)
        val resp_time_i = float(avg_table.iloc[i, 2])
        val util_i = float(avg_table.iloc[i, 1])
        total_resp_time += resp_time_i
        val max_util = max(max_util, util_i)
    
    println(f"n={n}: Total Response Time={total_resp_time:.3f}, Max Utilization={max_util:.3f}")